this is a test for transferring model from this dataset to predict labels in a private dataset.

**Import dataset and library**

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
test_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [ ]:
train_df

In [ ]:
#train_df.reset_index(drop=True)

In [ ]:
#before decreasing freqency
df_new=train_df.iloc[:,0:149]
df_new = df_new.loc[~df_new.index.duplicated(keep='first')]
df_new

In [ ]:
# pre-processing step: interpolation and smoothing
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline

In [ ]:
#plot ecg the first row

lst = [float(item) for item in df_new.iloc[0]]
array = np.reshape(lst, (-1,1))
plt.figure(figsize=(50,10))

plt.plot(array)
plt.ylabel('y_axis')
plt.xlabel('before decreasing frequency')
plt.show()

In [ ]:
len(df_new.columns)

In [ ]:
df_new.shape

In [ ]:
label=train_df[187]
label

In [ ]:
m =150
x, y  = df_new.shape
df_final = pd.concat([df_new, pd.DataFrame(np.zeros((x, m - y)), columns=range(y, m))], axis=1)
df_final[150]=label
df_final

In [ ]:
test_new=test_df.iloc[:,0:150]
test_new=test_new.loc[~test_new.index.duplicated(keep='first')]
test_new

In [ ]:
label_test=test_df[187]
label_test

In [ ]:
k =150
x, y  = test_new.shape
test_final = pd.concat([test_new, pd.DataFrame(np.zeros((x, k - y)), columns=range(y, k))], axis=1)
test_final[150]=label_test
test_final

In [ ]:
df_final[150]=df_final[150].astype(int)
equilibre=df_final[150].value_counts()
print(equilibre)

In [ ]:
from sklearn.utils import resample
df_1=df_final[df_final[150]==1]
df_2=df_final[df_final[150]==2]
df_3=df_final[df_final[150]==3]
df_4=df_final[df_final[150]==4]
df_0=(df_final[df_final[150]==0]).sample(n=20000,random_state=42)

df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)

train_df_new=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])

In [ ]:
train_df_new.reset_index(drop=True)

In [ ]:
equilibre=train_df_new[150].value_counts()
print(equilibre)


In [ ]:
plt.figure(figsize=(20,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(equilibre, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#original dataset first row
plt.plot(train_df.iloc[0,:149])

In [ ]:
#padding with 0 to the length of 200 to match with other dataset
#plt.plot(df_final.iloc[0,:150])

In [ ]:
c=train_df_new.groupby(150,group_keys=False).apply(lambda train_df_new : train_df_new.sample(1))

I take one sample per class and i store it in a datafrmae in order to have an exmeple. 

In [ ]:
 #plt.plot(c.iloc[0,:150]

In this part i will speak o n what i do to transform data. 

In [ ]:
def add_gaussian_noise(signal):
    noise=np.random.normal(0,0.05,150)
    return (signal+noise)


I use a fonction ( will depend of the version) where i add a noise to the data to generilize my train.

In [ ]:
tempo=c.iloc[0,:150]
bruiter=add_gaussian_noise(tempo)

plt.subplot(2,1,1)
plt.plot(c.iloc[0,:150])

plt.subplot(2,1,2)
plt.plot(bruiter)

plt.show()



In [ ]:
target_train=train_df_new[150]
target_test=test_final[150]
y_train=to_categorical(target_train)
y_test=to_categorical(target_test)

In [ ]:
X_train=train_df_new.iloc[:,:150].values
X_test=test_final.iloc[:,:150].values
#for i in range(len(X_train)):
#    X_train[i,:149]= add_gaussian_noise(X_train[i,:149])
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

In [ ]:
len(X_train[1])

**Network**

In [ ]:
!pip install keras-multi-head

In [ ]:

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import StandardScaler

import os
import math
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Dropout, Reshape, Conv1D, BatchNormalization, Activation, AveragePooling1D, GlobalAveragePooling1D, Lambda, Input, Concatenate, Add, UpSampling1D, Multiply
from keras.models import Model
from keras.objectives import mean_squared_error
from keras import backend as K
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from keras.initializers import random_normal
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import Callback
from keras import metrics

from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import keras
from keras_multi_head import MultiHead
from keras import layers,models
from keras import optimizers

# define model

model = models.Sequential()
model.add(layers.Conv1D(128, kernel_size=5, activation='relu', input_shape=(X_train.shape[1],1),padding='same'))
 
model.add(MultiHead([
    keras.layers.Conv1D(filters=64,kernel_size=3,padding='same'),
    keras.layers.Conv1D(filters=64,kernel_size=5, padding='same'),
    keras.layers.Conv1D(filters=64,kernel_size=7,padding='same'),
], name='Multi-CNNs'))
model.add(BatchNormalization())  
model.add(keras.layers.Flatten(name='Flatten'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu',name='Dense-1'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax', name='Dense-2'))
model.build()
model.summary()
n_obs, feature, depth = X_train.shape
batch_size=250

def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.75
    t = n_obs//(10000 * batch_size)  # every epoch we do n_obs/batch_size iteration
    lrate = initial_lrate * math.exp(-k*t)
    return lrate

lrate = LearningRateScheduler(exp_decay)
adam = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)

model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, 
          epochs=100, 
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          shuffle=True)

In [ ]:
def evaluate_model(history,X_test,y_test,model):
    scores = model.evaluate((X_test),y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    print(history)
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    target_names=['0','1','2','3','4']
    
    y_true=[]
    for element in y_test:
        y_true.append(np.argmax(element))
    prediction_proba=model.predict(X_test)
    prediction=np.argmax(prediction_proba,axis=1)
    cnf_matrix = confusion_matrix(y_true, prediction)
    

In [ ]:
evaluate_model(history,X_test,y_test,model)
y_pred=model.predict(X_test)

In [ ]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:

import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['N', 'S', 'V', 'F', 'Q'],normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()
